# VacationPy

In [ ]:
!jupyter nbextension enable --py gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

# Import API key
from api_keys import gkey
gmaps.configure(api_key=gkey)

# Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Cape Town,19,ZA,1595951511,58,-33.93,18.42,61.00,19.46
1,Bluff,100,NZ,1595951512,85,-46.60,168.33,43.00,11.72
2,Thompson,90,CA,1595951432,93,55.74,-97.86,59.00,5.82
3,Bambous Virieux,40,MU,1595951512,78,-20.34,57.76,71.60,8.05
4,Jalu,0,LY,1595951512,19,29.03,21.55,97.54,11.59


# Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
hotel_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
hotel_df = hotel_df.dropna(how='any')
hotel_df.reset_index(inplace=True)
del hotel_df['index']
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Castro,0,BR,1595951308,48,-24.79,-50.01,75.11,12.73
1,Lagoa,0,PT,1595951530,85,39.05,-27.98,74.32,8.95
2,Goderich,0,CA,1595951534,89,43.75,-81.72,75.99,12.91
3,Diego de Almagro,0,CL,1595951536,13,-26.37,-70.05,79.77,7.45
4,Novoye Leushino,0,RU,1595951544,85,56.81,40.51,70.02,4.36


# Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotellist = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": gkey
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Castro,0,BR,1595951308,48,-24.79,-50.01,75.11,12.73,Castro
1,Lagoa,0,PT,1595951530,85,39.05,-27.98,74.32,8.95,Santa Cruz Da Graciosa
2,Goderich,0,CA,1595951534,89,43.75,-81.72,75.99,12.91,Goderich
3,Diego de Almagro,0,CL,1595951536,13,-26.37,-70.05,79.77,7.45,Diego de Almagro
4,Novoye Leushino,0,RU,1595951544,85,56.81,40.51,70.02,4.36,Teykovo


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))